<a href="https://colab.research.google.com/github/benjaminbrown038/WeightAndBiases/blob/main/notebooks/wandb/visualize_predictions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Visualize Predictions With Pytorch

#### Imports

In [ ]:
%%capture
!pip3 install wandb -qqq
import wandb
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as T
import torch.nn.functional as F
wandb.login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.3 MB/s eta 0:00:00


#### Project Name

In [ ]:
WANDB_PROJECT = "mnist-viz"

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

#### Dataloader

In [ ]:
def get_dataloader(is_train, batch_size, slice=5):
    "Get a training dataloader"
    ds = torchvision.datasets.MNIST(root=".", train=is_train, transform=T.ToTensor(), download=True)
    loader = torch.utils.data.DataLoader(dataset=ds,batch_size=batch_size,shuffle=True if is_train else False,pin_memory=True, num_workers=2)
    return loader

#### Hyperparameters

In [ ]:
EPOCHS = 1
NUM_BATCHES_TO_LOG = 10
NUM_IMAGES_PER_BATCH = 32
NUM_CLASSES = 10
BATCH_SIZE = 32
LEARNING_RATE = 0.001
L1_SIZE = 32
L2_SIZE = 64
CONV_KERNEL_SIZE = 5

#### Model

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, L1_SIZE, CONV_KERNEL_SIZE, stride=1, padding=2),
            nn.BatchNorm2d(L1_SIZE),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(L1_SIZE, L2_SIZE, CONV_KERNEL_SIZE, stride=1, padding=2),
            nn.BatchNorm2d(L2_SIZE),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*L2_SIZE, NUM_CLASSES)
        self.softmax = nn.Softmax(NUM_CLASSES)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

100%|██████████| 9912422/9912422 [00:00<00:00, 78924689.66it/s]


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 72929376.17it/s]


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 22430524.27it/s]


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 4178663.91it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



#### Model

In [ ]:
train_loader = get_dataloader(is_train=True, batch_size=BATCH_SIZE)
test_loader = get_dataloader(is_train=False, batch_size=2*BATCH_SIZE)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#### Hyperparameters

In [ ]:
EPOCHS = 1
NUM_BATCHES_TO_LOG = 10
NUM_IMAGES_PER_BATCH = 32
NUM_CLASSES = 10
BATCH_SIZE = 32
LEARNING_RATE = 0.001
L1_SIZE = 32
L2_SIZE = 64
CONV_KERNEL_SIZE = 5

#### Model

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, L1_SIZE, CONV_KERNEL_SIZE, stride=1, padding=2),
            nn.BatchNorm2d(L1_SIZE),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(L1_SIZE, L2_SIZE, CONV_KERNEL_SIZE, stride=1, padding=2),
            nn.BatchNorm2d(L2_SIZE),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(7*7*L2_SIZE, NUM_CLASSES)
        self.softmax = nn.Softmax(NUM_CLASSES)
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

### Data

#### Training Data

In [ ]:
train_loader = get_dataloader(is_train=True, batch_size=BATCH_SIZE)
train_loader

#### Testing Data

In [ ]:
test_loader = get_dataloader(is_train=False, batch_size=2*BATCH_SIZE)
test_loader

#### Device

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

#### Initialize Project

In [ ]:
wandb.init(project="table-quickstart")

#### Configure Run

In [ ]:
cfg = wandb.config
cfg

#### Configure Update

In [ ]:
cfg.update({"epochs" : EPOCHS, "batch_size": BATCH_SIZE, "lr" : LEARNING_RATE,
            "l1_size" : L1_SIZE, "l2_size": L2_SIZE,
            "conv_kernel" : CONV_KERNEL_SIZE,
            "img_count" : min(10000, NUM_IMAGES_PER_BATCH*NUM_BATCHES_TO_LOG)})

#### Model

In [ ]:
model = ConvNet(NUM_CLASSES).to(device)
model

#### Criterion (Loss)

In [ ]:
criterion = nn.CrossEntropyLoss()
criterion

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
optimizer

In [ ]:
def log_test_predictions(images, labels, outputs, predicted, test_table, log_counter):
    scores = F.softmax(outputs.data, dim=1)
    log_scores = scores.cpu().numpy()
    log_images = images.cpu().numpy()
    log_labels = labels.cpu().numpy()
    log_preds = predicted.cpu().numpy()
    _id = 0
    for i, l, p, s in zip(log_images, log_labels, log_preds, log_scores):
        img_id = str(_id) + "_" + str(log_counter)
        test_table.add_data(img_id, wandb.Image(i), p, l, *s)
        _id += 1
        if _id == NUM_IMAGES_PER_BATCH:
        break
total_step = len(train_loader)
for epoch in range(EPOCHS):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        wandb.log({"loss" : loss})
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, EPOCHS, i+1, total_step, loss.item()))
    columns=["id", "image", "guess", "truth"]
    for digit in range(10):
        columns.append("score_" + str(digit))
    test_table = wandb.Table(columns=columns)
    model.eval()
    log_counter = 0
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            if log_counter < NUM_BATCHES_TO_LOG:
                log_test_predictions(images, labels, outputs, predicted, test_table, log_counter)
                log_counter += 1
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        acc = 100 * correct / total
        wandb.log({"epoch" : epoch, "acc" : acc})
        print('Test Accuracy of the model on the 10000 test images: {} %'.format(acc))
    wandb.log({"test_predictions" : test_table})
wandb.finish()

wandb: Currently logged in as: bkb20053. Use `wandb login --relogin` to force relogin


Epoch [1/1], Step [100/1875], Loss: 0.3033
Epoch [1/1], Step [200/1875], Loss: 0.0538
Epoch [1/1], Step [300/1875], Loss: 0.2059
Epoch [1/1], Step [400/1875], Loss: 0.0908
Epoch [1/1], Step [500/1875], Loss: 0.2653
Epoch [1/1], Step [600/1875], Loss: 0.2116
Epoch [1/1], Step [700/1875], Loss: 0.2680
Epoch [1/1], Step [800/1875], Loss: 0.0584
Epoch [1/1], Step [900/1875], Loss: 0.0627
Epoch [1/1], Step [1000/1875], Loss: 0.0132
Epoch [1/1], Step [1100/1875], Loss: 0.0065
Epoch [1/1], Step [1200/1875], Loss: 0.0300
Epoch [1/1], Step [1300/1875], Loss: 0.0367
Epoch [1/1], Step [1400/1875], Loss: 0.0603
Epoch [1/1], Step [1500/1875], Loss: 0.0154
Epoch [1/1], Step [1600/1875], Loss: 0.0793
Epoch [1/1], Step [1700/1875], Loss: 0.0055
Epoch [1/1], Step [1800/1875], Loss: 0.0052
Test Accuracy of the model on the 10000 test images: 98.53 %


acc,▁
epoch,▁
loss,█▄▄▂▁▂▃▂▁▂▁▁▁▂▂▁▁▁▁▁▁▁▃▁▂▁▂▁▁▁▁▁▁▁▁▁▃▁▁▁
acc,98.53
epoch,0
loss,0.00155
